# Migration der BE-Listen

In [1]:
import openpyxl
import pandas as pd
from natsort import index_natsorted
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import glob

In [2]:
# Einbandart Y
# max Öffnungswinkel AW
# dig mit Begleitung AY
# Testphase BM

def kommentare(bestand):
    wb = openpyxl.load_workbook(f"thomschke/be-{bestand}.xlsm")
    spalten_liste = ("Y", "AW", "AY", "BM")
    for spalte in spalten_liste:
        for cell in wb["Basis"][spalte]:
            if cell.comment:
                wb["Basis"].cell(row=cell.row, column=cell.column+1, value=cell.comment.text.split(":", maxsplit=1)[1])
    
    wb.save(f"thomschke/be-{bestand}-kommentare.xlsx")

In [12]:
def mergen_erstabgleich(bestand):
    gruppen = {"ii":"II", "böink":"Bö Ink", "böm": "Bö M", "schreibmeister":"Schreibmeister", "iii":"III","iv":"IV"}

    be = pd.read_excel(f"thomschke/be-{bestand}-kommentare.xlsx", sheet_name="Basis", dtype={"IDN":"string","AKZ":"string"})
    abzug = pd.read_csv(f"../abzug/{bestand}.csv", dtype={"idn":"string","akz":"string"})

    df = be.merge(abzug, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

    # idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
    df.loc[df["IDN"].isna(), "IDN"] = df.idn
    df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
    df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

    # wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
    df.loc[df["idn"].notna(), "digi"] = True
    df.loc[df["idn"].isna(), "digi"] = False
    df.fillna('', inplace=True)
    df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))
    df["Gruppe"] = gruppen[bestand]
    # die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
    spaltenliste = list(df.columns)[:-12]
    spaltenliste.insert(0, "digi")
    spaltenliste.insert(0, "Gruppe")
    spaltenliste.insert(4, "bbg")
    spaltenliste.insert(7, "signatur_g")

    wb = openpyxl.load_workbook(f"thomschke/be-{bestand}-kommentare.xlsx")
    wb.remove(wb["Basis"])
    ws = wb.create_sheet("Basis", 0)
    df = df.astype("object")
    for r in dataframe_to_rows(df[spaltenliste], index=False):
        ws.append(r)
    tab = Table(displayName="Basistabelle", ref=ws.dimensions)
    tab.tableStyleInfo = TableStyleInfo(name="Hell1", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    ws.add_table(tab)
    wb.save(f"listen_fin/{bestand}.xlsx")
    # df.to_excel(f"listen_fin/{bestand}-df.xlsx", columns=spaltenliste, index=False)

In [9]:
def mergen_zweitabgleich(bestand):
    gruppen = {"ii":"II", "böink":"Bö Ink", "böm": "Bö M", "schreibmeister":"Schreibmeister", "iii":"III","iv":"IV"}

    be = pd.read_excel(f"thomschke/be-{bestand}-kommentare.xlsx", sheet_name="Basis", dtype={"IDN":"string","AKZ":"string"})
    abzug = pd.read_csv(f"../abzug/{bestand}.csv", dtype={"idn":"string","akz":"string"})

    df = be.merge(abzug, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

    # idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
    df.loc[df["IDN"].isna(), "IDN"] = df.idn
    df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
    df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

    # wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
    df.loc[df["idn"].notna(), "digi"] = True
    df.loc[df["idn"].isna(), "digi"] = False
    df.fillna('', inplace=True)
    df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))
    df["Gruppe"] = gruppen[bestand]
    # die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
    spaltenliste = list(df.columns)[:-22]
    # spaltenliste.insert(0, "digi")
    # spaltenliste.insert(0, "Gruppe")
    # spaltenliste.insert(4, "bbg")
    # spaltenliste.insert(7, "signatur_g")

    wb = openpyxl.load_workbook(f"thomschke/be-{bestand}-kommentare.xlsx")
    wb.remove(wb["Basis"])
    ws = wb.create_sheet("Basis", 0)
    df = df.astype("object")
    for r in dataframe_to_rows(df[spaltenliste], index=False):
        ws.append(r)
    tab = Table(displayName="Basistabelle", ref=ws.dimensions)
    tab.tableStyleInfo = TableStyleInfo(name="Hell1", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    ws.add_table(tab)
    wb.save(f"listen_fin/{bestand}.xlsx")
    # df.to_excel(f"listen_fin/{bestand}-df.xlsx", columns=spaltenliste, index=False)

In [13]:
bestaende = ("ii", "böink", "böm", "schreibmeister", "iii", "iv")
# for bestand in bestaende:
#     kommentare(bestand)

for bestand in bestaende:
    mergen_erstabgleich(bestand)

/tmp/ipykernel_4195/4268170458.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Gruppe"] = gruppen[bestand]
/tmp/ipykernel_4195/4268170458.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Gruppe"] = gruppen[bestand]
/tmp/ipykernel_4195/4268170458.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

# Probleme

- [ ] falsche Signaturen an Fragmenten Böink in alter Liste
- mehrere Exemplare trotz 4105 belegt
- 1141698420 aufgelöster Sammelband, Verknüpfung aus 4105 raus
- Inkunabelfragmente II 90,1a etc. ?
- Tabellenformatierung übernehmen
- Link zum Portal reparieren
